# **This will take about 10 minutes.**

# **You Can Drink Coffe For A While.**

**# Please run in Colab**

In [ ]:
!pip install fiftyone
!pip uninstall opencv-python -y
!pip install opencv-python

# **開始下載資料**

In [3]:
import fiftyone as fo
import sys, gc, os
import psutil
import time

In [4]:
def ram():
  # result format is psutil.virtual_memory()
  result = psutil.virtual_memory()

  #     0    1       "2" ...
  name = ['Total', 'Available', 'Percent', 'Used', 'Free', 'Active', 'Inactive', 'Buffers', 'Cached', 'Shared', 'Slab']
  now_size=[]
  for i,i1 in zip(name, result):
    try:
      if(i=="Percent"):
        i1=format(i1,'.3f')
        print("{} : {:>2} %".format(i, i1)) 
      else:  
        print("{} : {:>2}".format(i, get_size(i1)))
    except Exception as e:
      print(e)      

def get_size(bytes_size):
  # Input is bytes
  byte=bytes_size
  out=0
  size = ["bytes", "KB", "MB", "GB", "TB"]
  for i in range(0,len(size)):
    byte=byte/1024
    if(abs(byte) < 1 ) :
      out="{:.3f} {}".format(byte*1024 ,size[i])
      break
  if(out==0):
     out="{:.3f} {}".format(bytes_size/(1024**4) ,size[4])    
  return out       

In [5]:
def count_time(fun1):

  def wrapper_function(*args, **kwargs):

    start_time = time.time()
    fun1_result=fun1(*args, **kwargs)
    start_time = time.time()-start_time
    times = ['ms', 's', 'Minutes', 'Hour']
    time_count = 0

    if(start_time<1):
      start_time*=1000
    else:
      time_count+=1
      while (start_time>=1 and time_count<len(times) ):
        start_time/=60
        time_count+=1
      start_time*=60  
      time_count-=1

    start_time = str(format(start_time ,'.3f'))+" "+times[time_count]
    print(start_time)
    return fun1_result
  return wrapper_function

In [ ]:
@count_time
def get_open_images(label_type,split_type,my_class,max_values):
  dataset = fo.zoo.load_zoo_dataset(
              "open-images-v6",
              split=split_type,
              label_types=[label_type],
              classes=[my_class],
              max_samples=max_values,
          )
"""
Copied on https://storage.googleapis.com/openimages/web/download.html

dataset = fiftyone.zoo.load_zoo_dataset(
              "open-images-v6",
              split="validation",
              label_types=["detections", "segmentations"],
              classes=["Cat", "Dog"],
              max_samples=100,
          )
"""

In [7]:
get_open_images("detections","train","Cat",100)

 100% |██████|    4.8Gb/4.8Gb [3.4s elapsed, 0s remaining, 1.3Gb/s]        
 100% |███████████████████| 100/100 [5.8s elapsed, 0s remaining, 16.5 files/s]      
Dataset info written to '/root/fiftyone/open-images-v6/info.json'
Loading 'open-images-v6' split 'train'
 100% |█████████████████| 100/100 [473.3ms elapsed, 0s remaining, 211.3 samples/s]      
Dataset 'open-images-v6-train-100' created
4.455Minutes


In [5]:
del fo

In [6]:
ram()

Total : 12.687 GB
Available : 11.484 GB
Percent : 9.500 %
Used : 1.004 GB
Free : 7.676 GB
Active : 1.715 GB
Inactive : 3.021 GB
Buffers : 129.004 MB
Cached : 3.881 GB
Shared : 1.156 MB
Slab : 219.270 MB


# **開始處理資料**

In [6]:
file_names=None

# 指定要列出所有檔案的目錄
mypath = "/root/fiftyone/open-images-v6/train/data"

for dirPath, dirNames, fileNames in os.walk(mypath):
    file_names=fileNames

for i in range(0,len(file_names)):
  file_names[i] = file_names[i].split('.')[0]    

In [7]:
def view_file(nam):
  with open(nam) as infile:  
    i = 0
    for line in infile:   
      print(line)
      if(i==2):
        break
      i+=1  

In [14]:
view_file('/root/fiftyone/open-images-v6/train/labels/detections.csv')

ImageID,Source,LabelName,Confidence,XMin,XMax,YMin,YMax,IsOccluded,IsTruncated,IsGroupOf,IsDepiction,IsInside,XClick1X,XClick2X,XClick3X,XClick4X,XClick1Y,XClick2Y,XClick3Y,XClick4Y

000002b66c9c498e,xclick,/m/01g317,1,0.012500,0.195312,0.148438,0.587500,0,1,0,0,0,0.148438,0.012500,0.059375,0.195312,0.148438,0.357812,0.587500,0.325000

000002b66c9c498e,xclick,/m/01g317,1,0.025000,0.276563,0.714063,0.948438,0,1,0,0,0,0.025000,0.248438,0.276563,0.214062,0.914062,0.714063,0.782813,0.948438



In [15]:
view_file('/root/fiftyone/open-images-v6/train/metadata/classes.csv')

/m/011k07,Tortoise

/m/011q46kg,Container

/m/012074,Magpie



In [ ]:
view_file('/root/fiftyone/open-images-v6/train/metadata/image_ids.csv')

In [22]:
csv_class = None
class_content = []
with open('/root/fiftyone/open-images-v6/train/metadata/classes.csv') as infile:  
   for i in infile:
     class_content.append(i.split('\n')[0].split(','))

In [24]:
for i in class_content:
  if(i[1]=='Cat'):
    print(i[0])
    break

/m/01yrx


In [67]:
csv_column = None
csv_content = []
with open('/root/fiftyone/open-images-v6/train/labels/detections.csv') as infile:  
  i = 0
  line = iter(infile)
  csv_column = next(line).split('\n')[0].split(',')
  print(csv_column)
  try: 
    while True:
      con = next(line).split('\n')[0].split(',')

      if( (con[0] in file_names) and (con[2]=='/m/01yrx') ):
        csv_content.append([con[0], con[2], con[4], con[5], con[6], con[7]])      
  except:
    pass    

['ImageID', 'Source', 'LabelName', 'Confidence', 'XMin', 'XMax', 'YMin', 'YMax', 'IsOccluded', 'IsTruncated', 'IsGroupOf', 'IsDepiction', 'IsInside', 'XClick1X', 'XClick2X', 'XClick3X', 'XClick4X', 'XClick1Y', 'XClick2Y', 'XClick3Y', 'XClick4Y']


In [ ]:
csv_content

In [ ]:
gc.collect()

52

Process datas

In [ ]:
!pwd

/content


In [51]:
from PIL import Image
from PIL import ImageColor
from PIL import ImageDraw
from PIL import ImageFont
from PIL import ImageOps
from IPython.display import display as ds
import numpy as np

In [114]:
def draw_bounding_box_on_image(image,
                               xmin,
                               xmax,
                               ymin,
                               ymax,
                               color,
                               font,
                               thickness=4,
                               display_str_list=()):
  """Adds a bounding box to an image."""
  draw = ImageDraw.Draw(image)
  im_width, im_height = image.size
  (left, right, top, bottom) = (xmin * im_width, xmax * im_width,
                                ymin * im_height, ymax * im_height)
  draw.line([(left, top), (left, bottom), (right, bottom), (right, top),
             (left, top)],
            width=thickness,
            fill=color)

  # If the total height of the display strings added to the top of the bounding
  # box exceeds the top of the image, stack the strings below the bounding box
  # instead of above.
  display_str_heights = [font.getsize(ds)[1] for ds in display_str_list]
  # Each display_str has a top and bottom margin of 0.05x.
  total_display_str_height = (1 + 2 * 0.05) * sum(display_str_heights)

  if top > total_display_str_height:
    text_bottom = top
  else:
    text_bottom = top + total_display_str_height
  # Reverse list and print from bottom to top.
  for display_str in display_str_list[::-1]:
    text_width, text_height = font.getsize(display_str)
    margin = np.ceil(0.05 * text_height)
    draw.rectangle([(left, text_bottom - text_height - 2 * margin),
                    (left + text_width, text_bottom)],
                   fill=color)
    draw.text((left + margin, text_bottom - text_height - margin),
              display_str,
              fill="black",
              font=font)
    text_bottom -= text_height - 2 * margin
  return image  
 
def draw_boxes(
     image, 
     class_names, 
     xmin,
     xmax,
     ymin,
     ymax):

  colors = list(ImageColor.colormap.values())

  font = ImageFont.load_default()
  display_str = (class_names)
  color = colors[hash(class_names) % len(colors)]
  image_pil = Image.open(image)
  image=draw_bounding_box_on_image(
        image_pil,
        xmin,
        xmax,
        ymin,
        ymax,
        color,
        font,
        display_str_list=[display_str])
  return image

def draw_boxes(
     image,
     class_names,
     xmin0, 
     xmax0, 
     ymin0, 
     ymax0):
  image = np.array(Image.open(image))
  now_image_np=np.zeros((image.shape))
  np.copyto(now_image_np , image)

  for i in range(0,len(xmin0)):
      class_name = class_names[i]
      xmin ,xmax, ymin, ymax = xmin0[i] ,xmax0[i], ymin0[i], ymax0[i]
      xmin ,xmax, ymin, ymax = [float(ii) for ii in [ xmin ,xmax, ymin, ymax ] ]
      colors = list(ImageColor.colormap.values())
    
      font = ImageFont.load_default()
      display_str = (class_name)
      color = colors[hash(class_name) % len(colors)]

      image_pil = Image.fromarray(np.uint8(now_image_np)).convert("RGB")

      draw_bounding_box_on_image(
            image_pil,
            xmin,
            xmax,
            ymin,
            ymax,
            color,
            font,
            display_str_list=[display_str])
    
      np.copyto(now_image_np, np.array(image_pil) )

  ds(Image.fromarray(np.uint8( np.array(now_image_np) )).convert("RGB") ) 

In [69]:
def get_label(Label, YouRange):
  Result = None
  for i in YouRange:
    if(Label==i[0]):
      Result = i[1]
      break
  return Result

def get_img_path(name, YouRange):
  Result = []
  for i in YouRange:
    if(name==i[0]):
      Result.append(i)
  return Result      

In [ ]:
my_data = get_img_path(file_names[2],csv_content)
my_path = "/root/fiftyone/open-images-v6/train/data/"+my_data[0][0]+".jpg"
my_label = [get_label( i[1], class_content) for i in my_data]
xmin = [i[2] for i in my_data]
xmax = [i[3] for i in my_data]
ymin = [i[4] for i in my_data]
ymax = [i[5] for i in my_data]
draw_boxes( my_path , 
      my_label ,
      xmin , 
      xmax , 
      ymin , 
      ymax 
      )

In [117]:
def show_image(myName):
  my_data = get_img_path(myName,csv_content)
  my_path = "/root/fiftyone/open-images-v6/train/data/"+my_data[0][0]+".jpg"
  my_label = [get_label( i[1], class_content) for i in my_data]
  xmin = [i[2] for i in my_data]
  xmax = [i[3] for i in my_data]
  ymin = [i[4] for i in my_data]
  ymax = [i[5] for i in my_data]
  draw_boxes( my_path , 
      my_label ,
      xmin , 
      xmax , 
      ymin , 
      ymax 
      )

In [ ]:
show_image(file_names[98])

In [ ]:
for i in file_names[25:30:1]:
  show_image(i)